In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment

schema = "sql_workshop_cloud"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD")
port = 3306
api_key = os.getenv("AEROBOX_API_KEY")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
airport_df= pd.read_sql("cities_airport", con=connection_string)


def icao_airport_codes(cities_df):
    icao_codes =  []
    city_id = []
    airport_name = []

    cities_df = get_cities(connection_string)

    for index, row in cities_df.iterrows():
        latitude = row['Latitude'] # be aware of column names in your SQL table
        longitude = row['Longitude'] # be aware of column names in your SQL table
        
        
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat": latitude,
                       "lon":longitude,
                       "radiusKm":"50",
                       "limit":"10",
                       "withFlightInfoOnly":"true"}

        headers = {
	        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": api_key #"YOUR_API_KEY_HERE"
        }

        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            airport_json = response.json()
        else:
            return response.reason
        
        # to append the data
        for data in airport_json['items']:
            icao_codes.append(data['icao'])
            city_id.append(row['City_id'])
            airport_name.append(data['name'])
            
        

    return pd.DataFrame({"city_id": city_id, "icao_codes": icao_codes, "airport_name" : airport_name })
